In [48]:
# import libraries

from utils import text_processing
from utils import get_weather
import datetime

import requests

import pandas as pd

from bs4 import BeautifulSoup

import re

import nltk
import os
import string
import numpy as np
import copy 
import pickle
import re
import math

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer



import pandas as pd

import scipy
from scipy import sparse

import numpy as np
from sklearn.model_selection import TimeSeriesSplit

import pickle

In [3]:
# get article from yesterday

#https://understandingwar.org/backgrounder/russian-offensive-campaign-assessment-april-23-2023

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

yesterday_day = yesterday.day
yesterday_month = yesterday.month
yesterday_year = yesterday.year


In [4]:
file = text_processing.get_article_from_yesterday(yesterday_day,yesterday_month,yesterday_year)

In [5]:
data = text_processing.read_html(file)

In [6]:
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

In [7]:
def preprocess_all_text(data):
    pattern = "\[(\d+)\]"


    data['main_html_v1'] = data['main_html'].apply(lambda x: re.sub(pattern,"",str(x)))
    data['main_html_v2'] = data['main_html_v1'].apply(lambda x: re.sub(r'http(\S+.*\s)',"",x))
    data['main_html_v3'] = data['main_html_v2'].apply(lambda x: re.sub(r'2022|2023|©2022|©2023|\xa0|\n',"",x))

    data['main_html_v4'] = data['main_html_v3'].apply(lambda x: BeautifulSoup(x).text)
    
    data['main_html_v5'] = data['main_html_v4'].apply(lambda x: text_processing.remove_names_and_dates(x))
    
    return data

In [8]:
data_preprocessed = preprocess_all_text(data)

In [9]:
#data_preprocessed.head()

In [10]:
data_preprocessed = data_preprocessed.drop(['main_html_v1','main_html_v2','main_html_v3','main_html_v4'],axis=1)

In [11]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikita_voitishyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
data_preprocessed['report_text_lemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"lemm"))
data_preprocessed['report_text_stemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"stemm"))

In [13]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm
0,2023-04-25,russian_offensive_campaign_assessment_April_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, April ...",russian offens campaign ass april twenty five...,russian offen campaign assess april twenti fi...


In [14]:
docs = data_preprocessed['report_text_lemm'].tolist()

In [15]:
#tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
#cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))

In [49]:
# CountVectorizer + tfidf:create a vecror with numbers

cv = CountVectorizer()
word_count_vector = cv.fit_transform(docs)

word_count_vector.shape

tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_transformer.fit(word_count_vector)

tf_idf_vector = tfidf_transformer.transform(word_count_vector)

#<340x7401 sparse matrix of type '<class 'numpy.float64'>'
#	with 213470 stored elements in Compressed Sparse Row format>

In [17]:
tf_idf_vector

<1x840 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [18]:
feature_names = cv.get_feature_names()
tf_idf_vector

<1x840 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [19]:
data_preprocessed['keywords'] = data_preprocessed['report_text_stemm'].apply(lambda x: text_processing.convert_doc_to_vector(x,feature_names,tf_idf_vector))

In [20]:
data_preprocessed['keywords'][0]

{'russian': 0.708,
 'forc': 0.275,
 'ukrainian': 0.245,
 'twenty': 0.168,
 'april': 0.153,
 'claim': 0.132,
 'militari': 0.127,
 'five': 0.122,
 'report': 0.107,
 'two': 0.092}

In [50]:
data_preprocessed.head()

date                                            short_url  \
0 2023-04-25  russian_offensive_campaign_assessment_April_25_2023   

                                                                                    title  \
0  Russian Offensive Campaign Assessment, April 25, 2023 | Institute for the Study of War   

                                              text_title  \
0  Russian Offensive Campaign Assessment, April 25, 2023   

                                                            full_url  \
0  /backgrounder/russian-offensive-campaign-assessment-april-25-2023   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

#### Part of scropt: 3_final_preprocess

In [21]:
data_preprocessed["date_datetime"] = pd.to_datetime(data_preprocessed["date"])
data_preprocessed['date_tomorrow_datetime'] = data_preprocessed['date_datetime'].apply(lambda x: x+datetime.timedelta(days=1))
data_preprocessed = data_preprocessed.rename(columns = {"date_datetime":"report_date"})

In [51]:
# погода на всі регіони треба: модифікувати скрипт get_weather?

region = "Донеччина"
today_date = "2023-04-25"

weather_for_12_hours = get_weather.get_weather_for_12_hours(region,today_date)

Weather data for the 
region Донеччина; 
start_date 2023-04-25; 
end_date 2023-04-25; 
is ready


In [23]:
weather_for_12_hours.head()

,datetime,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,...,pressure,visibility,cloudcover,solarradiation,solarenergy,uvindex,conditions,icon,stations,source
0,00:00:00,1682370000,51.7,51.7,96.09,50.6,0.031,100.0,0.0,0.0,...,1014.0,None,100.0,None,None,None,"Rain, Overcast",rain,[remote],obs
1,01:00:00,1682373600,51.2,51.2,89.84,48.3,0.024,100.0,0.0,0.0,...,1014.0,None,100.0,None,None,None,"Rain, Overcast",rain,[remote],obs
2,02:00:00,1682377200,50.3,50.3,90.41,47.6,0.020,100.0,0.0,0.0,...,1013.0,None,93.5,None,None,None,"Rain, Overcast",rain,[remote],obs
3,03:00:00,1682380800,49.5,49.0,91.61,47.2,0.008,100.0,0.0,0.0,...,1012.0,None,98.4,None,None,None,"Rain, Overcast",rain,[remote],obs
4,04:00:00,1682384400,49.4,48.8,90.99,46.8,0.000,0.0,0.0,0.0,...,1011.0,None,98.7,None,None,None,Overcast,cloudy,[remote],obs


In [24]:
weather_for_12_hours['date'] = today_date
weather_for_12_hours["day_datetime"] = pd.to_datetime(weather_for_12_hours["date"])

In [25]:
weather_for_12_hours.columns

Index(['datetime', 'datetimeEpoch', 'temp', 'feelslike', 'humidity', 'dew',
       'precip', 'precipprob', 'snow', 'snowdepth', 'preciptype', 'windgust',
       'windspeed', 'winddir', 'pressure', 'visibility', 'cloudcover',
       'solarradiation', 'solarenergy', 'uvindex', 'conditions', 'icon',
       'stations', 'source', 'date', 'day_datetime'],
      dtype='object')

In [26]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords,report_date,date_tomorrow_datetime
0,2023-04-25,russian_offensive_campaign_assessment_April_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, April ...",russian offens campaign ass april twenty five...,russian offen campaign assess april twenti fi...,"{'russian': 0.708, 'forc': 0.275, 'ukrainian':...",2023-04-25,2023-04-26


In [27]:
data_preprocessed_short = df_isw_short = data_preprocessed[['date','date_tomorrow_datetime','keywords','report_text_lemm']]

In [28]:
data_preprocessed_short.head()

,date,date_tomorrow_datetime,keywords,report_text_lemm
0,2023-04-25,2023-04-26,"{'russian': 0.708, 'forc': 0.275, 'ukrainian':...",russian offens campaign ass april twenty five...


In [29]:
data_merged = weather_for_12_hours.merge(data_preprocessed_short,
                         how="left", left_on = "day_datetime", right_on = "date_tomorrow_datetime")

In [30]:
data_merged['solarradiation']

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
Name: solarradiation, dtype: object

In [31]:
tf_idf_vector

<1x840 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [32]:
data_merged = data_merged[['day_datetime','temp', 'feelslike', 'humidity', 'dew',
       'precip', 'precipprob', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'pressure', 'cloudcover']]

In [33]:
data_merged.head()

,day_datetime,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,windgust,windspeed,winddir,pressure,cloudcover
0,2023-04-25,51.7,51.7,96.09,50.6,0.031,100.0,0.0,0.0,10.1,3.8,90.8,1014.0,100.0
1,2023-04-25,51.2,51.2,89.84,48.3,0.024,100.0,0.0,0.0,10.7,4.7,77.9,1014.0,100.0
2,2023-04-25,50.3,50.3,90.41,47.6,0.020,100.0,0.0,0.0,10.3,3.1,60.8,1013.0,93.5
3,2023-04-25,49.5,49.0,91.61,47.2,0.008,100.0,0.0,0.0,6.7,3.1,50.6,1012.0,98.4
4,2023-04-25,49.4,48.8,90.99,46.8,0.000,0.0,0.0,0.0,6.7,3.1,30.9,1011.0,98.7


In [34]:
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   day_datetime  24 non-null     datetime64[ns]
 1   temp          24 non-null     float64       
 2   feelslike     24 non-null     float64       
 3   humidity      24 non-null     float64       
 4   dew           24 non-null     float64       
 5   precip        24 non-null     float64       
 6   precipprob    24 non-null     float64       
 7   snow          24 non-null     float64       
 8   snowdepth     24 non-null     float64       
 9   windgust      24 non-null     float64       
 10  windspeed     24 non-null     float64       
 11  winddir       24 non-null     float64       
 12  pressure      24 non-null     float64       
 13  cloudcover    24 non-null     float64       
dtypes: datetime64[ns](1), float64(13)
memory usage: 2.8 KB


In [35]:
#data_merged.isnull().sum()

day_datetime    0
temp            0
feelslike       0
humidity        0
dew             0
precip          0
precipprob      0
snow            0
snowdepth       0
windgust        0
windspeed       0
winddir         0
pressure        0
cloudcover      0
dtype: int64

In [36]:
# треба доробити матрицю!!!

#weather_alarms_tfidf_csr = scipy.sparse.csr_matrix(data_merged)
#weather_alarms_tfidf_features = scipy.sparse.hstack((weather_alarms_tfidf_csr, tf_idf_vector), format="csr")

In [37]:
data_vectorised = tf_idf_vector.toarray()
vectors_df = pd.DataFrame(data_vectorised)
vectors_df['date'] = pd.to_datetime(today)


In [38]:
vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,831,832,833,834,835,836,837,838,839,date
0,0.005094,0.005094,0.015281,0.005094,0.005094,0.005094,0.010188,0.005094,0.010188,0.005094,...,0.005094,0.030563,0.005094,0.015281,0.015281,0.005094,0.010188,0.005094,0.025469,2023-04-26


In [39]:
data_merged = data_merged.rename(columns = {"day_datetime":"date"})
data_merged.head()

,date,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,windgust,windspeed,winddir,pressure,cloudcover
0,2023-04-25,51.7,51.7,96.09,50.6,0.031,100.0,0.0,0.0,10.1,3.8,90.8,1014.0,100.0
1,2023-04-25,51.2,51.2,89.84,48.3,0.024,100.0,0.0,0.0,10.7,4.7,77.9,1014.0,100.0
2,2023-04-25,50.3,50.3,90.41,47.6,0.020,100.0,0.0,0.0,10.3,3.1,60.8,1013.0,93.5
3,2023-04-25,49.5,49.0,91.61,47.2,0.008,100.0,0.0,0.0,6.7,3.1,50.6,1012.0,98.4
4,2023-04-25,49.4,48.8,90.99,46.8,0.000,0.0,0.0,0.0,6.7,3.1,30.9,1011.0,98.7


In [40]:
merged_df = pd.merge(data_merged, vectors_df,on='date')

In [41]:
merged_df.head()

,date,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,windgust,...,830,831,832,833,834,835,836,837,838,839


### PREDICT

In [42]:
merged_df = merged_df.drop(columns=['date'])

In [43]:
model = pickle.load(open("models/4_rf_3.1f.pkl","rb"))

/Users/nikita_voitishyn/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/nikita_voitishyn/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [44]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 853 entries, temp to 839
dtypes: float64(853)
memory usage: 0.0+ bytes


In [45]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [46]:
print(data_merged.columns)

for column in data_merged.columns:
    print(column)

Index(['date', 'temp', 'feelslike', 'humidity', 'dew', 'precip', 'precipprob',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure',
       'cloudcover'],
      dtype='object')
date
temp
feelslike
humidity
dew
precip
precipprob
snow
snowdepth
windgust
windspeed
winddir
pressure
cloudcover


In [47]:
model.predict(merged_df)

ValueError: Found array with 0 sample(s) (shape=(0, 853)) while a minimum of 1 is required.